In [3]:
#Generic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from tqdm import tqdm
import os
import time
import datetime


In [4]:
## pytorch packages (not needed here)
from tqdm import auto
import torch
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_networkx

import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GCNConv, global_mean_pool, TopKPooling, SAGPooling, GATConv
from torch_geometric.nn import HANConv

c:\Users\Joe\miniconda3\envs\ptorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
game_data = pd.read_csv('aggr_team_data.csv')
player_data = pd.read_csv('aggr_player_data_full.csv')
team_team_data = pd.read_csv('edge_features.csv')

In [17]:
print(len(team_team_data))

8460


In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
game_data.head()

,Team_ID,Game_ID,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,...,AVG_TOV,AVG_PF,AVG_PTS,W,L,W_PCT,W_PCT_PREV_10,IS_HOME_TEAM,HOME_TEAM_WINS,OPPONENT_ID
0,1610612737,21300012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1,0.000,0.000000,0,0,1610612749
1,1610612737,21300023,37.000000,76.0,0.487000,8.00,24.00,0.333000,27.000000,35.000000,...,17.000000,20.00,109.000000,1,1,0.500,0.000000,1,1,1610612766
2,1610612737,21300046,36.500000,76.5,0.477500,9.00,23.50,0.384000,23.500000,33.000000,...,15.000000,16.00,105.500000,1,2,0.333,0.500000,0,0,1610612748
3,1610612737,21300059,37.666667,81.0,0.466333,9.00,23.00,0.392333,20.333333,30.666667,...,13.666667,18.00,104.666667,2,2,0.500,0.333333,0,0,1610612751
4,1610612737,21300071,38.000000,81.0,0.470000,8.75,22.75,0.385250,20.000000,29.500000,...,13.750000,17.25,104.750000,2,3,0.400,0.500000,0,0,1610612738


In [20]:
### drop the first 10 games of each team

### start by making a list of each game_id that is one of the first 10 games of a team

first_10_games = []

game_data['game_number'] = game_data['W'] + game_data['L']

first_10_games = game_data[game_data["game_number"] <= 10]

first_10_games_ids = first_10_games["Game_ID"].unique().tolist()




In [21]:
first_10_games_ids.append(22000016)

In [22]:
### filter out these games from our data:

# Filter out rows with GAME_ID in first_10_games_ids for game_data
filtered_game_data = game_data[~game_data["Game_ID"].isin(first_10_games_ids)]

filtered_edge_data = team_team_data[~team_team_data["Game_ID"].isin(first_10_games_ids)]


In [23]:

# Filter out rows with GAME_ID in first_10_games_ids for player_data
filtered_player_data = player_data[~player_data["Game_ID"].isin(first_10_games_ids)]


In [24]:
player_data.head()

,PLAYER_ID,Game_ID,Team_ID,AVG_MIN_x,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,...,AVG_USG_PCT,AVG_E_USG_PCT,AVG_E_PACE,AVG_PACE,AVG_PACE_PER40,AVG_POSS,AVG_PIE,START_POSITION,DID_NOT_PLAY,BIRTHDATE
0,708,21300004,1610612751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,0,2330
1,708,21300028,1610612751,26.283333,4.000000,8.00,0.500000,0.0,0.000000,0.0,...,0.179000,0.17700,97.320000,94.980000,79.150000,52.000000,0.118,F,0,2330
2,708,21300043,1610612751,26.041667,3.500000,9.50,0.386500,0.0,0.000000,0.0,...,0.210000,0.21300,97.815000,98.655000,82.210000,54.000000,0.068,F,0,2330
3,708,21300052,1610612751,24.833333,3.333333,10.00,0.348667,0.0,0.333333,0.0,...,0.232333,0.23500,98.743333,99.316667,82.763333,51.333333,0.059,F,0,2330
4,708,21300077,1610612751,22.025000,3.000000,8.75,0.361500,0.0,0.250000,0.0,...,0.232000,0.23525,96.435000,96.987500,80.822500,44.750000,0.076,F,0,2330


In [25]:
#Drop rows with NaN values
filtered_player_data = filtered_player_data.dropna()

#Drop rows where DID_NOT_PLAY has the value 1 -- ignoring players who don't play in the game
filtered_player_data = filtered_player_data[filtered_player_data["DID_NOT_PLAY"] != 1]

print(len(filtered_player_data))

83694


In [26]:
####Check edge features for NA rows
print(len(filtered_edge_data))
print(len(filtered_game_data))

7365
14730


In [27]:
#One-hot encode START_POSITION
start_position_dummies = pd.get_dummies(filtered_player_data['START_POSITION'], prefix='START_POSITION')

#Concatenate the one-hot encoded columns to the original DataFrame
filtered_player_data = pd.concat([filtered_player_data, start_position_dummies], axis=1)

#Drop the original START_POSITION column
filtered_player_data = filtered_player_data.drop('START_POSITION', axis=1)


In [28]:
##MinMaxScale GAME DATA
from sklearn.preprocessing import MinMaxScaler

#List the columns you want to scale in filtered_game_data
#Make sure to exclude any non-numeric columns and ID columns
numeric_columns = filtered_game_data.columns.difference(['Team_ID', 'Game_ID', 'OPPONENT_ID'])

#Create a MinMaxScaler instance
scaler = MinMaxScaler()

#Fit and transform the selected columns
filtered_game_data[numeric_columns] = scaler.fit_transform(filtered_game_data[numeric_columns])


C:\Users\Joe\AppData\Local\Temp\ipykernel_25956\2505614329.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_game_data[numeric_columns] = scaler.fit_transform(filtered_game_data[numeric_columns])


In [29]:
filtered_game_data.head()

,Team_ID,Game_ID,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,...,AVG_PF,AVG_PTS,W,L,W_PCT,W_PCT_PREV_10,IS_HOME_TEAM,HOME_TEAM_WINS,OPPONENT_ID,game_number
10,1610612737,21300158,0.502825,0.409159,0.622066,0.312848,0.308810,0.486996,0.398047,0.408212,...,0.183949,0.464417,0.082192,0.069444,0.545,0.117647,0.0,0.0,1610612764,0.000000
11,1610612737,21300166,0.464532,0.375304,0.599209,0.331871,0.311086,0.529472,0.356692,0.368338,...,0.208409,0.421061,0.095890,0.069444,0.583,0.098039,1.0,1.0,1610612757,0.014085
12,1610612737,21300180,0.449885,0.335454,0.620607,0.302515,0.289509,0.489976,0.353820,0.367680,...,0.156723,0.396831,0.109589,0.069444,0.615,0.117647,0.0,0.0,1610612750,0.028169
13,1610612737,21300190,0.442803,0.351866,0.594647,0.301522,0.290512,0.485874,0.336809,0.357102,...,0.127772,0.382918,0.109589,0.083333,0.571,0.117647,1.0,0.0,1610612756,0.042254
14,1610612737,21300213,0.431800,0.392534,0.545115,0.284063,0.284665,0.449421,0.304177,0.320117,...,0.185322,0.352634,0.109589,0.097222,0.533,0.117647,1.0,0.0,1610612761,0.056338


In [30]:
##MinMax Scale Player Data
numeric_columns = filtered_player_data.columns.difference(['PLAYER_ID', 'Game_ID', 'Team_ID'])

scaler = MinMaxScaler()


filtered_player_data[numeric_columns] = scaler.fit_transform(filtered_player_data[numeric_columns])

In [31]:
filtered_player_data.head()

,PLAYER_ID,Game_ID,Team_ID,AVG_MIN_x,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,...,AVG_E_PACE,AVG_PACE,AVG_PACE_PER40,AVG_POSS,AVG_PIE,DID_NOT_PLAY,BIRTHDATE,START_POSITION_C,START_POSITION_F,START_POSITION_G
9,708,21300161,1610612751,0.543178,0.230159,0.407789,0.327778,0.0,0.016712,0.0,...,0.132621,0.016240,0.016243,0.530776,0.570053,0.0,0.748933,0.0,1.0,0.0
10,708,21300182,1610612751,0.538532,0.221429,0.375258,0.395000,0.0,0.015041,0.0,...,0.131888,0.015642,0.015644,0.524318,0.575495,0.0,0.748933,0.0,1.0,0.0
11,708,21300197,1610612751,0.533786,0.227273,0.363636,0.419727,0.0,0.013674,0.0,...,0.132633,0.015802,0.015804,0.522269,0.579509,0.0,0.748933,0.0,1.0,0.0
12,708,21300211,1610612751,0.533875,0.220238,0.364261,0.403250,0.0,0.018801,0.0,...,0.132463,0.015803,0.015805,0.521550,0.577273,0.0,0.748933,0.0,1.0,0.0
13,708,21300218,1610612751,0.537782,0.230769,0.371134,0.407231,0.0,0.017355,0.0,...,0.131279,0.015327,0.015330,0.521854,0.579144,0.0,0.748933,0.0,1.0,0.0


In [32]:
###### need to create edge features between the two teams based on their previous games against each other, subtracting the away team's stats from the home team's stats
###             check that one ^

### now

filtered_edge_data.head()


,Team_ID,Game_ID,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,...,PREV_MATCHUP_DIFF_OREB,PREV_MATCHUP_DIFF_DREB,PREV_MATCHUP_DIFF_REB,PREV_MATCHUP_DIFF_AST,PREV_MATCHUP_DIFF_STL,PREV_MATCHUP_DIFF_BLK,PREV_MATCHUP_DIFF_TOV,PREV_MATCHUP_DIFF_PF,PREV_MATCHUP_DIFF_PTS,OPPONENT_ID
4,1610612765,21300166,38.100000,84.100000,0.454400,6.100000,21.100000,0.277500,17.200000,24.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612738
5,1610612738,21300190,36.214286,81.571429,0.447000,5.428571,16.785714,0.319786,14.714286,19.071429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612765
6,1610612753,21300213,37.538462,81.846154,0.460615,8.076923,20.769231,0.383077,15.769231,21.307692,...,-2.0,5.0,3.0,14.0,0.0,4.0,-5.0,-9.0,10.0,1610612754
7,1610612742,21300231,39.250000,83.687500,0.471438,8.937500,23.000000,0.387062,18.062500,22.312500,...,-2.0,-7.0,-9.0,-4.0,9.0,3.0,-3.0,-2.0,-9.0,1610612739
8,1610612746,21300270,39.000000,82.166667,0.476056,8.111111,23.666667,0.337778,20.333333,28.777778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612741


In [33]:
##MinMaxScale edge _data

#List the columns you want to scale in filtered_game_data
#Make sure to exclude any non-numeric columns and ID columns
numeric_columns = filtered_edge_data.columns.difference(['Team_ID', 'Game_ID', 'OPPONENT_ID'])


scaler = MinMaxScaler()


filtered_edge_data[numeric_columns] = scaler.fit_transform(filtered_edge_data[numeric_columns])


C:\Users\Joe\AppData\Local\Temp\ipykernel_25956\1901224814.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edge_data[numeric_columns] = scaler.fit_transform(filtered_edge_data[numeric_columns])


In [34]:
filtered_edge_data[filtered_edge_data['AVG_FGM'].isna()]

,Team_ID,Game_ID,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,...,PREV_MATCHUP_DIFF_OREB,PREV_MATCHUP_DIFF_DREB,PREV_MATCHUP_DIFF_REB,PREV_MATCHUP_DIFF_AST,PREV_MATCHUP_DIFF_STL,PREV_MATCHUP_DIFF_BLK,PREV_MATCHUP_DIFF_TOV,PREV_MATCHUP_DIFF_PF,PREV_MATCHUP_DIFF_PTS,OPPONENT_ID


In [35]:
filtered_game_data[filtered_game_data['Game_ID'] == 22000016]

,Team_ID,Game_ID,AVG_FGM,AVG_FGA,AVG_FG_PCT,AVG_FG3M,AVG_FG3A,AVG_FG3_PCT,AVG_FTM,AVG_FTA,...,AVG_PF,AVG_PTS,W,L,W_PCT,W_PCT_PREV_10,IS_HOME_TEAM,HOME_TEAM_WINS,OPPONENT_ID,game_number


In [36]:
filtered_edge_data.isna().sum()

Team_ID                         0
Game_ID                         0
AVG_FGM                         0
AVG_FGA                         0
AVG_FG_PCT                      0
AVG_FG3M                        0
AVG_FG3A                        0
AVG_FG3_PCT                     0
AVG_FTM                         0
AVG_FTA                         0
AVG_FT_PCT                      0
AVG_OREB                        0
AVG_DREB                        0
AVG_REB                         0
AVG_AST                         0
AVG_STL                         0
AVG_BLK                         0
AVG_TOV                         0
AVG_PF                          0
AVG_PTS                         0
W                            7361
L                            7361
W_PCT                        7361
W_PCT_PREV_10                   0
IS_HOME_TEAM                    0
HOME_TEAM_WINS                  0
PREV_MATCHUP_DIFF_FGM           0
PREV_MATCHUP_DIFF_FGA           0
PREV_MATCHUP_DIFF_FG_PCT        0
PREV_MATCHUP_D

In [41]:
#### Now we need to make the graph data including edges between players and teams and players and players of the same position



In [37]:
print(len(filtered_player_data))
filtered_player_data.columns

83694


Index(['PLAYER_ID', 'Game_ID', 'Team_ID', 'AVG_MIN_x', 'AVG_FGM', 'AVG_FGA',
       'AVG_FG_PCT', 'AVG_FG3M', 'AVG_FG3A', 'AVG_FG3_PCT', 'AVG_FTM',
       'AVG_FTA', 'AVG_FT_PCT', 'AVG_OREB', 'AVG_DREB', 'AVG_REB', 'AVG_AST',
       'AVG_STL', 'AVG_BLK', 'AVG_TO', 'AVG_PF', 'AVG_PTS', 'AVG_PLUS_MINUS',
       'AVG_E_OFF_RATING', 'AVG_OFF_RATING', 'AVG_E_DEF_RATING',
       'AVG_DEF_RATING', 'AVG_E_NET_RATING', 'AVG_NET_RATING', 'AVG_AST_PCT',
       'AVG_AST_TOV', 'AVG_AST_RATIO', 'AVG_OREB_PCT', 'AVG_DREB_PCT',
       'AVG_REB_PCT', 'AVG_TM_TOV_PCT', 'AVG_EFG_PCT', 'AVG_TS_PCT',
       'AVG_USG_PCT', 'AVG_E_USG_PCT', 'AVG_E_PACE', 'AVG_PACE',
       'AVG_PACE_PER40', 'AVG_POSS', 'AVG_PIE', 'DID_NOT_PLAY', 'BIRTHDATE',
       'START_POSITION_C', 'START_POSITION_F', 'START_POSITION_G'],
      dtype='object')

In [38]:
print(len(filtered_game_data))
filtered_game_data.columns

14730


Index(['Team_ID', 'Game_ID', 'AVG_FGM', 'AVG_FGA', 'AVG_FG_PCT', 'AVG_FG3M',
       'AVG_FG3A', 'AVG_FG3_PCT', 'AVG_FTM', 'AVG_FTA', 'AVG_FT_PCT',
       'AVG_OREB', 'AVG_DREB', 'AVG_REB', 'AVG_AST', 'AVG_STL', 'AVG_BLK',
       'AVG_TOV', 'AVG_PF', 'AVG_PTS', 'W', 'L', 'W_PCT', 'W_PCT_PREV_10',
       'IS_HOME_TEAM', 'HOME_TEAM_WINS', 'OPPONENT_ID', 'game_number'],
      dtype='object')

In [39]:
print(len(filtered_edge_data))
filtered_edge_data.columns

7365


Index(['Team_ID', 'Game_ID', 'AVG_FGM', 'AVG_FGA', 'AVG_FG_PCT', 'AVG_FG3M',
       'AVG_FG3A', 'AVG_FG3_PCT', 'AVG_FTM', 'AVG_FTA', 'AVG_FT_PCT',
       'AVG_OREB', 'AVG_DREB', 'AVG_REB', 'AVG_AST', 'AVG_STL', 'AVG_BLK',
       'AVG_TOV', 'AVG_PF', 'AVG_PTS', 'W', 'L', 'W_PCT', 'W_PCT_PREV_10',
       'IS_HOME_TEAM', 'HOME_TEAM_WINS', 'PREV_MATCHUP_DIFF_FGM',
       'PREV_MATCHUP_DIFF_FGA', 'PREV_MATCHUP_DIFF_FG_PCT',
       'PREV_MATCHUP_DIFF_FG3M', 'PREV_MATCHUP_DIFF_FG3A',
       'PREV_MATCHUP_DIFF_FG3_PCT', 'PREV_MATCHUP_DIFF_FTM',
       'PREV_MATCHUP_DIFF_FTA', 'PREV_MATCHUP_DIFF_FT_PCT',
       'PREV_MATCHUP_DIFF_OREB', 'PREV_MATCHUP_DIFF_DREB',
       'PREV_MATCHUP_DIFF_REB', 'PREV_MATCHUP_DIFF_AST',
       'PREV_MATCHUP_DIFF_STL', 'PREV_MATCHUP_DIFF_BLK',
       'PREV_MATCHUP_DIFF_TOV', 'PREV_MATCHUP_DIFF_PF',
       'PREV_MATCHUP_DIFF_PTS', 'OPPONENT_ID'],
      dtype='object')

In [40]:
import torch
from torch_geometric.data import HeteroData

def create_game_hetero_data(game_id, player_data, game_data, edge_data):
    #Initialize dictionaries to store node and edge features
    node_features = {
        'team': [],
        'player': []
    }
    edge_index = {
        ('team', 'team'): [],
        ('player', 'player'): [],
        ('player', 'team'): []
    }
    edge_features = {
        ('team', 'team'): []
    }

    player_dict = {} ### a mapping of the player node ID to the player ID
    home_away_dict = {}  ### mapping of home and away to team IDs
    team_dict = {} ### mapping of team node ID to team ID
    
    home_team_node_id = game_data[(game_data['Game_ID'] == game_id) & (game_data['IS_HOME_TEAM'] == 1)]['Team_ID'].values[0]
    away_team_node_id = game_data[(game_data['Game_ID'] == game_id) & (game_data['IS_HOME_TEAM'] == 0)]['Team_ID'].values[0]

    #Add team nodes and features
    #print('Adding team nodes and features')
    game_teams = game_data[game_data['Game_ID'] == game_id]
    for _, row in game_teams.iterrows():
        team_node_id = row['Team_ID']
        if 0 in team_dict.keys():
            team_dict[1] = team_node_id
        else:
            team_dict[0] = team_node_id
        if row['IS_HOME_TEAM'] == 1:
            home_away_dict['Home'] = team_node_id
        else:
            home_away_dict['Away'] = team_node_id
        node_features['team'].append(row.drop(['Team_ID', 'Game_ID', 'OPPONENT_ID', 'W', 'L', 'HOME_TEAM_WINS']).to_numpy())
        
    #Add edge index and features between the two teams
    #print('Adding edge index and features between the two teams')
    edge_index[('team', 'team')].append((0, 1))
    edge_features[('team', 'team')].append(edge_data[edge_data['Game_ID'] == game_id].drop(['Team_ID', 'Game_ID', 'OPPONENT_ID', 'W', 'L', 'W_PCT' ,'IS_HOME_TEAM','HOME_TEAM_WINS'], axis=1).to_numpy().squeeze())
        

    #Add player nodes and features, and edges between players and their teams
    #print('Adding player nodes and features, and edges between players and their teams')
    game_players = player_data[player_data['Game_ID'] == game_id]
    #print(game_players.columns)
    counter = 0
    for i, row in game_players.iterrows():
        player_node_id = row['PLAYER_ID']
        player_dict[counter] = player_node_id
        node_features['player'].append(row.drop(['PLAYER_ID', 'Game_ID', 'Team_ID','DID_NOT_PLAY']).to_numpy())
        team_node_id = row['Team_ID']
        edge_index[('player', 'team')].append((int(counter), int(list(team_dict.keys())[list(team_dict.values()).index(team_node_id)]) ))
        counter += 1
        
    #Add edges between players who play the same position but on opposite teams
    #print('Adding edges between players who play the same position but on opposite teams')
    seen = []
    for id1, player_row1 in game_players.iterrows():
        seen.append(id1)
        for id2, player_row2 in game_players.iterrows():
            if id2 not in seen:
                if player_row1['Team_ID'] != player_row2['Team_ID'] and (player_row1[['START_POSITION_C', 'START_POSITION_F', 'START_POSITION_G']] == player_row2[['START_POSITION_C', 'START_POSITION_F', 'START_POSITION_G']]).all():
                    edge_index[('player', 'player')].append((int(list(player_dict.keys())[list(player_dict.values()).index(player_row1['PLAYER_ID'])]), int(list(player_dict.keys())[list(player_dict.values()).index(player_row2['PLAYER_ID'])])))

    #HeteroData object
    game_hetero_data = HeteroData()
    game_hetero_data['team'].x = torch.tensor(node_features['team'], dtype=torch.float)
    game_hetero_data['player'].x = torch.tensor(node_features['player'], dtype=torch.float)
    game_hetero_data[('team', 'team')].edge_index = torch.tensor(edge_index[('team', 'team')], dtype=torch.long).t().contiguous()
    game_hetero_data[('team', 'team')].edge_attr = torch.tensor(edge_features[('team', 'team')], dtype=torch.float)
    game_hetero_data[('player', 'player')].edge_index = torch.tensor(edge_index[('player', 'player')], dtype=torch.long).t().contiguous()
    game_hetero_data[('player', 'team')].edge_index = torch.tensor(edge_index[('player', 'team')], dtype=torch.long).t().contiguous()
    
    # graph meta data
    game_hetero_data['game_id'] = game_id
    game_hetero_data['player_dict'] = player_dict
    game_hetero_data['home_away_dict'] = home_away_dict
    game_hetero_data['team_dict'] = team_dict



    #Add the label based on HOME_TEAM_WINS
    home_team_row = game_teams[game_teams['IS_HOME_TEAM'] == 1].iloc[0]
    game_hetero_data.y = torch.tensor([home_team_row['HOME_TEAM_WINS']], dtype=torch.long)

    return game_hetero_data


In [41]:
game_ids = filtered_game_data['Game_ID'].unique()


In [42]:

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
    #### Create all the graphs

    hetero_data_list = [create_game_hetero_data(game_id, filtered_player_data, filtered_game_data, filtered_edge_data) for game_id in tqdm(game_ids, desc='Creating HeteroData objects')]


Creating HeteroData objects:   0%|          | 0/7365 [00:00<?, ?it/s]C:\Users\Joe\AppData\Local\Temp\ipykernel_25956\1958943942.py:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  game_hetero_data['team'].x = torch.tensor(node_features['team'], dtype=torch.float)
Creating HeteroData objects: 100%|██████████| 7365/7365 [06:26<00:00, 19.05it/s]


In [43]:
import pickle

# Serialize the list to a file
with open('all_games.pickle', 'wb') as f:
    pickle.dump(hetero_data_list, f)
    


In [106]:
# Deserialize the list from the file
with open('all_games.pickle', 'rb') as f:
    hetero_data_list = pickle.load(f)


In [ ]:
#from sklearn.model_selection import train_test_split
#foor the first run through with the model, we should use game_ids from the 6 seasons and then test on the 7th season, so we will need a mapping of game_ids to seasons
#game_ids = filtered_game_data['Game_ID'].unique()
#import original game_data files to get the dates for each game id:
#####


In [44]:
hetero_data_list[0].edge_index_dict

{('team',
  'to',
  'team'): tensor([[0],
         [1]]),
 ('player',
  'to',
  'player'): tensor([[0, 0, 1, 1, 2, 3, 3, 4, 6],
         [5, 8, 5, 8, 9, 4, 7, 6, 7]]),
 ('player',
  'to',
  'team'): tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
         [1, 1, 1, 1, 0, 0, 1, 0, 0, 0]])}

In [45]:
# Replace with the path to your CSV files
csv_folder = "C:\\Users\\Joe\\Desktop\\Data Science Masters\\Tufts Spring 2023\\CS 150\\Final Project\\Input Data\\team"


# Process all CSV files and store results in a list
game_dates = []

for file_name in os.listdir(csv_folder):
    if file_name.startswith("team_season_stats_") and file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
        season_data = pd.read_csv(file_path)
        for _, row in season_data.iterrows():
            game_dates.append((row['Game_ID'], row['GAME_DATE']))
        

# Combine all dates into one DataFrame
game_dates = pd.DataFrame(game_dates, columns=['Game_ID', 'GAME_DATE'])


In [46]:
import datetime as dt
#convert the GAME_DATE column to datetime objects
game_dates['GAME_DATE'] = pd.to_datetime(game_dates['GAME_DATE'])


In [47]:
game_dates.sort_values(by=['GAME_DATE'], inplace=True)

In [48]:

game_dates = game_dates[game_dates['Game_ID'].isin(game_ids)]

In [49]:
game_dates.drop_duplicates( inplace=True)
len(game_dates)

7365

In [63]:
len(game_dates[game_dates['GAME_DATE'] > dt.datetime(2021,3,31)])

375

In [64]:
375/7365

0.05091649694501019

In [15]:
572/8460 ### consider shrinking the test set

0.06761229314420804

In [66]:
train_game_ids = game_dates[game_dates['GAME_DATE'] <= dt.datetime(2021,3,31)]['Game_ID'].values
test_game_ids = game_dates[game_dates['GAME_DATE'] > dt.datetime(2021,3,31)]['Game_ID'].values

print(len(train_game_ids)+len(test_game_ids))

7365


In [67]:
train_graphs = [graph for graph in hetero_data_list if graph['game_id'] in train_game_ids]
test_graphs = [graph for graph in hetero_data_list if graph['game_id'] in test_game_ids]

print(len(train_graphs)+len(test_graphs))

7365


In [68]:
with open('train_graphs.pickle', 'wb') as f:
    pickle.dump(train_graphs, f)

with open('test_graphs.pickle', 'wb') as f:
    pickle.dump(test_graphs, f)

In [64]:
test = {0 : 14, 1: 19}

In [79]:
list(test.keys())[list(test.values()).index(19)]

1

In [69]:
train_graphs[0]

HeteroData(
  game_id=21300158,
  player_dict={
    0=2203.0,
    1=2544.0,
    2=2547.0,
    3=2592.0,
    4=2594.0,
    5=201143.0,
    6=201596.0,
    7=201952.0,
    8=201960.0,
    9=202970.0
  },
  home_away_dict={
    Away=1610612737.0,
    Home=1610612748.0
  },
  team_dict={
    0=1610612737.0,
    1=1610612748.0
  },
  y=[1],
  team={ x=[2, 22] },
  player={ x=[10, 46] },
  (team, to, team)={
    edge_index=[2, 1],
    edge_attr=[1, 37]
  },
  (player, to, player)={ edge_index=[2, 9] },
  (player, to, team)={ edge_index=[2, 10] }
)

In [72]:
train_graphs[0].edge_index_dict[('player','to','team' )]

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [1, 1, 1, 1, 0, 0, 1, 0, 0, 0]])